In [ ]:
# importing necessary dependencies
import pandas as pd
from datetime import datetime, timedelta

: 

In [ ]:
zulo_bank = pd.read_csv('Dataset/zulo_bank.csv')

In [ ]:
zulo_bank.head(5)

In [ ]:
zulo_bank.info()

In [ ]:
# clean dataset
zulo_bank.fillna({
    'LoanAmount': 0.0,
    'LoanType': 'Unknown',
    'InterestRate': 0.0
}, inplace=True)

In [ ]:
# convert fullName into firstName and lastName
zulo_bank[['FirstName', 'LastName']] = zulo_bank['FullName'].str.split(expand=True)
zulo_bank.head(5)

In [ ]:
# convert 1NF to 2NF
# Create Customer table
customer = zulo_bank[['FirstName', 'LastName', 'Email', 'Phone']].copy().drop_duplicates().reset_index(drop=True)
customer['Customer_id'] = range(1, len(customer) + 1)
customer = customer[['Customer_id', 'FirstName', 'LastName', 'Email', 'Phone']]

customer.head(5)

In [ ]:
# Create account table
account = zulo_bank[['AccountType', 'Balance', 'OpeningDate']].copy().drop_duplicates().reset_index(drop=True)
account['Account_id'] = range(1, len(account) + 1)
account = account[['Account_id', 'AccountType', 'Balance', 'OpeningDate']]

account.head(5)

In [ ]:
# Create transaction table
transaction = zulo_bank[['TransactionType', 'Amount', 'TransactionDate']].copy().drop_duplicates().reset_index(drop=True)
transaction['Transaction_id'] = range(1, len(transaction) + 1)
transaction = transaction[['Transaction_id', 'TransactionType', 'Amount', 'TransactionDate']]

transaction.head(5)

In [ ]:
# Create Loan table
loan = zulo_bank[['LoanType', 'LoanAmount', 'StartDate', 'EndDate', 'InterestRate']].copy().drop_duplicates().reset_index(drop=True)
loan['Loan_id'] = range(1, len(loan) + 1)
loan = loan[['Loan_id', 'LoanType', 'LoanAmount', 'StartDate', 'EndDate', 'InterestRate']]

loan.head(5)

In [ ]:
# Merge all tables to create a Zulo bank table
zulo_bank.merge(customer, on=['FirstName', 'LastName', 'Email', 'Phone'], how='left') \
         .merge(account, on=['AccountType', 'Balance', 'OpeningDate'], how='left') \
         .merge(transaction, on=['TransactionType', 'Amount', 'TransactionDate'], how='left') \
         .merge(loan, on=['LoanAmount', 'LoanType', 'StartDate', 'EndDate', 'InterestRate'], how='left') \
         [['Customer_id', 'Account_id', 'Transaction_id', 'Loan_id']]


In [ ]:
# convert 2NF to 3NF
# Create the date dimension table
# define the start and end date
start_date = datetime(2020, 1, 1)
end_date = datetime(2090, 12, 31)

# calculate the number of days between start and end date
num_days = (end_date - start_date).days

# generate a list of dates from start to current date
date_list = [start_date + timedelta(days=x) for x in range(num_days + 1)]

# Ensure date_id matches the length of date_list
date = {'date_id': [x for x in range(1, len(date_list) + 1)], 'date': date_list}

# Create Date DataFrame
date_dim = pd.DataFrame(date)
date_dim['day'] = date_dim['date'].dt.day
date_dim['month'] = date_dim['date'].dt.month
date_dim['year'] = date_dim['date'].dt.year
date_dim['date'] = pd.to_datetime(date_dim['date']).dt.date

date_dim.head(5)

In [ ]:
# Make sure both are datetime.date
account['OpeningDate'] = pd.to_datetime(account['OpeningDate']).dt.date
date_dim['date'] = pd.to_datetime(date_dim['date']).dt.date  # convert to date only

# Now merge
account = (
    account.merge(date_dim, left_on='OpeningDate', right_on='date', how='inner')
           .rename(columns={'date_id': 'OpeningDate_ID'})
           .reset_index(drop=True)
           [['Account_id', 'AccountType', 'Balance', 'OpeningDate_ID']]
)
account.head(5)

In [ ]:
# Transaction table 2NF to 3NF
transaction['TransactionDate'] = pd.to_datetime(transaction['TransactionDate']).dt.date
transaction = (
    transaction.merge(date_dim, left_on='TransactionDate', right_on='date', how='inner')\
               .rename(columns={'date_id': 'TransactionDate_ID'})\
               .reset_index(drop=True)\
               [['Transaction_id', 'TransactionType', 'Amount', 'TransactionDate_ID']]
)
transaction.head(5)

In [ ]:
zulo_bank.columns

In [ ]:
loan['StartDate'] = pd.to_datetime(
    loan['StartDate'].replace("Unknown", pd.NA), 
    errors='coerce'
).dt.date

loan['EndDate'] = pd.to_datetime(
    loan['EndDate'].replace("Unknown", pd.NA), 
    errors='coerce'
).dt.date

loan = loan.dropna(subset=['StartDate', 'EndDate'])

loan = (
    loan.merge(date_dim, left_on='StartDate', right_on='date', how='inner')
        .rename(columns={'date_id': 'StartDate_ID'})
        .merge(date_dim, left_on='EndDate', right_on='date', how='inner', suffixes=('', '_end'))
        .rename(columns={'date_id': 'EndDate_ID'})
        [['Loan_id', 'LoanAmount', 'LoanType', 'StartDate_ID', 'EndDate_ID', 'InterestRate']]
)
loan.head(5)

In [ ]:
# Save the tables as CSV files
transaction.to_csv(r'Database_model/transaction.csv', index=False)
loan.to_csv(r'Database_model/loan.csv', index=False)
account.to_csv(r'Database_model/account.csv', index=False)
customer.to_csv(r'Database_model/customer.csv', index=False)
date_dim.to_csv(r'Database_model/date_dim.csv', index=False)
zulo_bank.to_csv(r'Database_model/zulo_bank_cleaned.csv', index=False)

#### Transaction DWH Schema

In [ ]:
transaction_dim = transaction[['Transaction_id', 'TransactionType', ]].copy().drop_duplicates().reset_index(drop=True)
account_dim = account[['Account_id', 'AccountType', 'Account_Balance']].copy().drop_duplicates().reset_index(drop=True)

transaction_fact_table = zulo_bank.merge(transaction, on='Transaction_id', how='inner') \
                                  .merge(account, on='Account_id', how='inner') \
                                  [['Transaction_id', 'Account_id', 'OpeningDate_ID', 'TransactionDate_ID', 'Amount']]